Testing

In [ ]:
import cv2
import numpy as np
import pickle
import os
import random
from tensorflow.keras.utils import to_categorical

In [ ]:
import shutil

In [ ]:
shutil.unpack_archive("/content/train_testANGLE.zip", "/content/")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
DATADIR = '/content/train_testANGLE'
CATEGORIES = ['HIGH','MED','LOW']
training_data = []

In [ ]:
for category in CATEGORIES:
  path = os.path.join(DATADIR,category)
  class_num=CATEGORIES.index(category)
  for img in os.listdir(path):
    try:
      img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
      #img_array = img_array.ravel()
      training_data.append([img_array,class_num])

    except Exception as e:
      print('error')
      pass

In [ ]:
random.shuffle(training_data)

X = []
y = []

In [ ]:
for features, label in training_data:
  X.append(features)
  y.append(label)

In [ ]:
y=to_categorical(y)

In [ ]:
X = np.array(X).reshape(-1,320,180,1)
y = np.array(y)

In [ ]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y,pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pickle
import numpy as np

In [ ]:
X = pickle.load(open("X.pickle","rb"))
y = pickle.load(open("y.pickle","rb"))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
model = keras.Sequential([
  keras.layers.Conv2D(128,(3,3),activation='relu',input_shape=X_train.shape[1:]),
  keras.layers.MaxPool2D(2,2),
  keras.layers.Conv2D(64,(3,3),activation='relu'),
  keras.layers.MaxPool2D(2,2),

  keras.layers.Flatten(),
  keras.layers.Dense(64,activation='relu'),
  keras.layers.Dense(32,activation='relu'),
  keras.layers.Dropout(0.4),
  keras.layers.Dense(3,activation='softmax')
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 318, 178, 128)     1280      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 159, 89, 128)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 157, 87, 64)       73792     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 78, 43, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 214656)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                13738048  
_________________________________________________________________
dense_4 (Dense)              (None, 32)               

In [ ]:
from keras.optimizers import SGD
opt = SGD(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=3, epochs=15, validation_data=(X_test, y_test))

Epoch 1/15
202/202 [==============================] - 10s 47ms/step - loss: 0.6507 - accuracy: 0.7881 - val_loss: 0.7797 - val_accuracy: 0.7500
Epoch 2/15
202/202 [==============================] - 9s 47ms/step - loss: 0.8045 - accuracy: 0.7914 - val_loss: 0.7572 - val_accuracy: 0.7500
Epoch 3/15
202/202 [==============================] - 9s 47ms/step - loss: 0.6519 - accuracy: 0.7897 - val_loss: 0.7857 - val_accuracy: 0.7500
Epoch 4/15
202/202 [==============================] - 9s 47ms/step - loss: 0.5786 - accuracy: 0.7964 - val_loss: 0.8049 - val_accuracy: 0.7500
Epoch 5/15
202/202 [==============================] - 9s 47ms/step - loss: 0.5508 - accuracy: 0.8046 - val_loss: 0.8443 - val_accuracy: 0.7434
Epoch 6/15
202/202 [==============================] - 9s 47ms/step - loss: 0.5135 - accuracy: 0.7997 - val_loss: 1.1685 - val_accuracy: 0.7500
Epoch 7/15
202/202 [==============================] - 10s 47ms/step - loss: 0.4907 - accuracy: 0.8179 - val_loss: 0.9022 - val_accuracy: 0.72

In [ ]:
model.evaluate(X_test,y_test,verbose=0)

[2.2516963481903076, 0.6907894611358643]

In [ ]:
model.save("TEST_ANGLE.h5")


In [ ]:
from keras.models import load_model
angle = load_model("TEST_ANGLE.h5")


In [ ]:
%cd /content/test_angle

/content/test_angle


In [ ]:
import cv2
def prepare(filepath):
  new_img=cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)    
  new_img=cv2.resize(new_img, (160,90))    
  return new_img.reshape(-1, 160, 90,1)

In [ ]:
files = os.listdir("/content/test_angle")
CATEGORIES= ["High","Med","Low"]
#prediction = angle.predict(files[0])
#print(prediction)
#return position of max
for file in files[1:]:
  img = cv2.imread(file,cv2.IMREAD_GRAYSCALE)
  img = cv2.resize(img,(320,180))
  new_img = img.reshape(-1,320,180,1)
  prediction = angle.predict_classes(new_img)
  print(file)
  print(prediction)
  print(CATEGORIES[int(prediction)]) 

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


kns-cog1-Scene-1019-02.jpg
[2]
Low
knives.out.bdrip-no1-Scene-0048-01.jpg
[1]
Med
Do.the.Right.Thing.1989.HUN.DVDRip.XviD-pocokmaster-Scene-050-01.jpg
[1]
Med
Phantom.Thread.2017.BDRip.x264-GECKOS-Scene-159-02.jpg
[2]
Low
thefavourite-sd-hyperx-Scene-243-02.jpg
[2]
Low
the.last.black.man.in.san.francisco.2019.limited.bdrip.x264-cadaver-Scene-067-01.jpg
[2]
Low
